<a href="https://colab.research.google.com/github/ykitaguchi77/Instance_segmentation/blob/main/Train_YOLOv11_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train YOLOv11_segmentation (local)**

In [ ]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

### Train

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11 segment model
model = YOLO("yolo11x-seg.pt")

# Train the model (eyelid, iris, pupil)
yaml_path = r"C:\Users\CorneAI\Desktop\Mobius_single_annot\pupil\eyesegmentation.yaml"
results = model.train(data=yaml_path, epochs=600, imgsz=640)


### Resume

In [ ]:
from ultralytics import YOLO
# Resume training
# Load a model
model = YOLO(r"C:\Users\CorneAI\YOLOv11_Mobius\instanceenv\runs\segment\train15\weights\last.pt")  # load a partially trained model
results = model.train(resume=True)

In [ ]:
"""
project/
│
├── data/
│   ├── train/
│   │   ├── images/
│   │   └── labels/
│   │
│   ├── val/
│   │   ├── images/
│   │   └── labels/
│   │
│   └── test/  (オプション)
│       ├── images/
│       └── labels/
│
│
├── dataset.yaml
│
├── models/
│
└── runs/  (トレーニング結果が保存される)

#**Inference**

https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

#image_path = r"C:\CorneAI\DemoImage\典型例\【正常】demo008.BMP"
#image_path = r"C:\Users\CorneAI\YOLOv11_Mobius\samples\sample_1.jpg"
image_path = r"C:\Users\CorneAI\YOLOv11_Mobius\Segmentation_test_images\内斜視\5409-20030521-1-083545_9f2969d8ae990d14e09de27a0378a196f684c66f54658d24944400ec365229d1.JPG"
#image_path = r"C:\Users\CorneAI\YOLOv11_Mobius\Segmentation_test_images\先天性眼瞼下垂\93-20170215-3-134156_1aa2bdb042b6ecd2749d0f93d0831ead1e35e5267d50ae4d8d5c382816377408.jpg"

model = YOLO(r"C:\Users\CorneAI\YOLOv11_Mobius\instanceenv\runs\segment\train7\weights\best.pt")  # load a pretrained model (recommended for training)
#model = YOLO(r"C:\Users\CorneAI\YOLOv11_Mobius\models\best_n_458epochs.pt")  # load a pretrained model (recommended for training)

results = model(image_path,save=False)

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

### 座標の表示のみ

In [ ]:
from ultralytics import YOLO

# モデルとイメージの読み込み
image_path = r"C:\Users\CorneAI\YOLOv11_Mobius\samples\sample_2.jpg"
model = YOLO(r"C:\Users\CorneAI\YOLOv11_Mobius\models\best_458epochs.pt")
results = model(image_path, save=False)

# 各検出結果に対してセグメントの座標を表示
for result in results:
    # マスクが存在する場合
    if result.masks is not None:
        masks = result.masks  # Masks objectを取得

        # xy座標を取得（ピクセル座標）
        segments = masks.xy  # リストのリストとして座標を取得

        # 各セグメントに対して処理
        for i, segment in enumerate(segments):
            print(f"Segment {i + 1}:")
            print("Coordinates (x, y):")
            # 座標点を表示
            for point in segment:
                print(f"({point[0]:.1f}, {point[1]:.1f})")
            print("-" * 50)

### 並べて表示


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_single_image(image_path, model_path):
    # モデルの読み込み
    model = YOLO(model_path)

    # 画像の読み込み
    original_img = cv2.imread(image_path)
    if original_img is None:
        print(f"Failed to load image: {image_path}")
        return

    # 推論の実行
    results = model(original_img)

    # マスクの描画された画像を取得（ラベルなし）
    predicted_img = results[0].plot(labels=False, boxes=False)

    # 画像のサイズを揃える
    h1, w1 = original_img.shape[:2]
    h2, w2 = predicted_img.shape[:2]

    # 大きい方のサイズに合わせる
    max_h = max(h1, h2)
    max_w = max(w1, w2)

    # パディング
    padded_original = np.zeros((max_h, max_w, 3), dtype=np.uint8)
    padded_predicted = np.zeros((max_h, max_w, 3), dtype=np.uint8)

    # 元画像をパディングした画像に配置
    padded_original[:h1, :w1] = original_img
    padded_predicted[:h2, :w2] = predicted_img

    # 画像を横に連結
    combined_img = np.hstack((padded_original, padded_predicted))

    # Matplotlib で表示
    plt.figure(figsize=(20, 10))
    plt.imshow(cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# パラメータの設定
model_path = r"C:\Users\CorneAI\YOLOv11_Mobius\models\best_x_600epochs.pt"
image_path = r"C:\Users\CorneAI\YOLOv11_Mobius\samples\sample_2.jpg"

# 実行
display_single_image(image_path, model_path)